In [ ]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

from sklearn.preprocessing import LabelEncoder

In [ ]:
# load data and keeping the needed data columns 
data = pd.read_csv('/content/Sentiment.csv')
data = data[['text', 'sentiment']]
data.head(5) # looking at the data

,text,sentiment
0,RT @NancyLeeGrahn: How did everyone feel about...,Neutral
1,RT @ScottWalker: Didn't catch the full #GOPdeb...,Positive
2,RT @TJMShow: No mention of Tamir Rice and the ...,Neutral
3,RT @RobGeorge: That Carly Fiorina is trending ...,Positive
4,RT @DanScavino: #GOPDebate w/ @realDonaldTrump...,Positive


In [ ]:
data['text'] = data['text'].apply(lambda x: x.lower()) # lowercase text
data['text'] = data['text'].apply((lambda x: re.sub('[^a-zA-z0-9\s]', '', x)))

for idx, row in data.iterrows(): # iterate through the data and replace the rows with rt with ' ' 
  row[0] = row[0].replace('rt', ' ')

In [ ]:
max_features = 2000 # creating max feauture variable 
tokenizer = Tokenizer(num_words=max_features, split=' ') #passing in the number of max features and splitting the data

 # getting the values fitting the tokenizer 
tokenizer.fit_on_texts(data['text'].values) 
X = tokenizer.texts_to_sequences(data['text'].values) 
X = pad_sequences(X)

In [ ]:
embed_dim = 128 # embedding dimensions
lstm_out = 196 # lst output
# model created
def createmodel():
    model = Sequential()
    model.add(Embedding(max_features, embed_dim,input_length = X.shape[1]))
    model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
    model.add(Dense(3,activation='softmax'))
    model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
    model.save('model.h5') # saving the model 

    return model

model_sum = createmodel()
model_sum.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 28, 128)           256000    
_________________________________________________________________
lstm (LSTM)                  (None, 196)               254800    
_________________________________________________________________
dense (Dense)                (None, 3)                 591       
Total params: 511,391
Trainable params: 511,391
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# label encoding the senteiment columns 
labelencoder = LabelEncoder() 
integer_encoded = labelencoder.fit_transform(data['sentiment'])
y = to_categorical(integer_encoded)
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42) # traning the data with test size of 0.33

# compiling the model 
batch_size = 32
model = createmodel()
model.fit(X_train, Y_train, epochs = 1, batch_size=batch_size, verbose = 2)
score,acc = model.evaluate(X_test,Y_test,verbose=2,batch_size=batch_size)
# looking at the score
print(score)
print(acc)
print(model.metrics_names)

291/291 - 33s - loss: 0.8224 - accuracy: 0.6458
144/144 - 1s - loss: 0.7809 - accuracy: 0.6737
0.7808601260185242
0.6736566424369812
['loss', 'accuracy']


In [ ]:
from keras.wrappers.scikit_learn import KerasClassifier

model_2 = KerasClassifier(build_fn=createmodel, verbose=1)
batch_size = [10, 20, 40]
epochs = [1, 2, 3]
param_grid = dict (batch_size=batch_size, epochs=epochs)
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model_2, param_grid=param_grid)
grid_result = grid.fit(X_train, Y_train)

print("Best: %f using %s"%(grid_result.best_score_, grid_result.best_params_))

186/186 [==============================] - 2s 8ms/step - loss: 0.7752 - accuracy: 0.6690
Epoch 1/2
744/744 [==============================] - 84s 110ms/step - loss: 0.8790 - accuracy: 0.6247
Epoch 2/2
186/186 [==============================] - 2s 9ms/step - loss: 0.7238 - accuracy: 0.6799
Epoch 1/2
744/744 [==============================] - 86s 113ms/step - loss: 0.8758 - accuracy: 0.6277
Epoch 2/2
186/186 [==============================] - 2s 9ms/step - loss: 0.7370 - accuracy: 0.6826
Epoch 1/2
744/744 [==============================] - 83s 109ms/step - loss: 0.8836 - accuracy: 0.6209
Epoch 2/2
186/186 [==============================] - 2s 9ms/step - loss: 0.7659 - accuracy: 0.6703
Epoch 1/2
744/744 [==============================] - 85s 110ms/step - loss: 0.8828 - accuracy: 0.6215
Epoch 2/2
186/186 [==============================] - 2s 9ms/step - loss: 0.7630 - accuracy: 0.6636
Epoch 1/2
744/744 [==============================] - 83s 109ms/step - loss: 0.8706 - accuracy: 0.6249
Epoch